In [1]:
% matplotlib notebook
import matplotlib.pyplot as plt
import numpy as np
import pandas
import torch, torch.utils.data, torchvision
import PIL
import os.path
import time
import skimage, skimage.io
import time
import copy
from my_utils import *

In [2]:
MEAN = [0.485, 0.456, 0.406] # expected by pretrained resnet18
STD = [0.229, 0.224, 0.225] # expected by pretrained resnet18

# Full model

In [3]:
# load the data and add a column for occurrences
df = pandas.read_csv('./data/train_no_duplicates.csv')
grouped = df.groupby('Id',group_keys=False)
df['occurrences'] =  grouped.Id.transform('count')

In [4]:
# For all of the whales which occur at least 11 times but ar not 'new_whale', take one random sample
tmp = df[(df.occurrences >+ 11) & (df.Id != 'new_whale')].groupby('Id',group_keys=False).apply(lambda x: x.sample(1,random_state=1234))
# Get the numeric indices of these samples
val_indices = tmp.index
# Append the indices for all the 'new_whales'
val_indices = val_indices.append(df[df.Id == 'new_whale'].index)

In [5]:
# Double check that all the indices are unique
print len(val_indices.unique()) == len(val_indices)

True


In [6]:
# Define the training indices as all the other indices
train_indices = df.index
train_indices = train_indices.drop(val_indices)

In [7]:
# Have a look at what the validation set looks like
df.loc[val_indices]

,Unnamed: 0,Image,Id,Hash_val,Copies,occurrences
6041,2666,45880deb.jpg,w_0e737d0,c5de92c1edb21269,1,17
3138,9446,f5ac7c84.jpg,w_1287fbc,fdc0f66ba80f8246,2,29
8730,9001,ea561a0e.jpg,w_17ee910,ec399e96d216c625,1,15
7849,9235,f0b8e93c.jpg,w_18eee6e,ea1a87e192a7b1e4,1,17
2967,5330,8ae4e714.jpg,w_1eafe46,ad90e32dac2bc33a,1,23
5017,3976,685c3732.jpg,w_2d99a0c,fa85a06f1dc2e439,2,12
1731,7546,c38ce869.jpg,w_43be268,bfca9265291b2a93,1,21
8266,8804,e53d2b96.jpg,w_4b7b80b,a9b7c6c82d37d0c8,1,13
3697,4920,7fce2c3f.jpg,w_4e52a49,abd694691e82e17c,2,12
7068,1406,24ef92ce.jpg,w_540fd73,b48ac73468dba569,1,14


In [8]:
# Define DataSamplers for train and validation set
sampler = {'train': torch.utils.data.sampler.SubsetRandomSampler(train_indices.tolist()),\
           'val': torch.utils.data.sampler.SubsetRandomSampler(val_indices.tolist())}

In [9]:
# Define transformations for the train and validation set, 
# with data augmentation and without data augmentation, respectively.

transforms = {'train': torchvision.transforms.Compose([
    torchvision.transforms.RandomRotation(degrees=20.0), # Data augmentation
    torchvision.transforms.RandomGrayscale(), # Data augmentation
    torchvision.transforms.Resize((224,224)),
    torchvision.transforms.ToTensor(), # Expected by pretrained neural network
    torchvision.transforms.Normalize(MEAN, STD) # Expected by pretrained neural network
    ]),\
            'val': torchvision.transforms.Compose([
    torchvision.transforms.Resize((224,224)),
    torchvision.transforms.ToTensor(), # Expected by pretrained neural network
    torchvision.transforms.Normalize(MEAN, STD) # Expected by pretrained neural network
    ])}

In [19]:
# In the WhaleDataset I used iloc, so now I need to reindex the dataframe
full_data = {x: WhaleDataset(df.reindex(columns=['Image','Id']),'./data/',transform=transforms[x]) for x in ['train', 'val']}

In [20]:
full_dataloader = {x: torch.utils.data.DataLoader(full_data[x],\
                                            batch_size=4,\
                                            shuffle=False,\
                                            num_workers=4,\
                                            sampler=sampler[x]) for x in ['train', 'val']}

In [25]:
print len(full_dataloader['val'].dataset.categories)

4246


In [21]:
# load the toy_model
toy_model = torch.load('./toy_model_all_layers_trained.pt')
print toy_model.fc

Linear(in_features=512, out_features=56, bias=True)


In [22]:
# Create the full model from the toy nodel by replacing the last layer
full_model = copy.deepcopy(toy_model)

# freeze layers
for params in full_model.parameters():
    params.requires_grad == False

# Create new final layer (unfrozen by default)
num_in = full_model.fc.in_features
num_out = len(full_dataloader['train'].dataset.categories)
full_model.fc = torch.nn.Linear(num_in,num_out)


# define the device
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
full_model.to(device)

print full_model.fc

Linear(in_features=512, out_features=4246, bias=True)


In [26]:
# Choose the loss function (criterion) and optimizer.
# I make the same choice as for the toy_model.
criterion = torch.nn.CrossEntropyLoss()
full_optimizer = torch.optim.Adam(full_model.fc.parameters(),lr=0.001)

In [9]:
# train the full model
full_model, loss_vals, acc_vals = train_with_restart(
    full_model,full_dataloader,criterion,full_optimizer,device,\
    use_val=False,num_epochs=1,T_max=15)

Epoch 0 / 30
Training: Elapsed time: 3174.1572 Loss: 9.6565 Acc: 0.0836
Epoch 1 / 30
Training: Elapsed time: 6329.4689 Loss: 5.8781 Acc: 0.0925
Epoch 2 / 30
Training: Elapsed time: 9489.7782 Loss: 5.0812 Acc: 0.1217
Epoch 3 / 30
Training: Elapsed time: 12652.6803 Loss: 4.4482 Acc: 0.1659
Epoch 4 / 30
Training: Elapsed time: 15814.9887 Loss: 3.8642 Acc: 0.2216
Epoch 5 / 30
Training: Elapsed time: 18964.6980 Loss: 3.3309 Acc: 0.2917
Epoch 6 / 30
Training: Elapsed time: 22129.9876 Loss: 2.8245 Acc: 0.3695
Epoch 7 / 30
Training: Elapsed time: 25306.8060 Loss: 2.4689 Acc: 0.4515
Epoch 8 / 30
Training: Elapsed time: 28516.4583 Loss: 2.1332 Acc: 0.5112
Epoch 9 / 30
Training: Elapsed time: 31795.9411 Loss: 1.8834 Acc: 0.5743
Epoch 10 / 30
Training: Elapsed time: 35083.2242 Loss: 1.7264 Acc: 0.6052
Epoch 11 / 30
Training: Elapsed time: 38348.0572 Loss: 3.8454 Acc: 0.2318
Epoch 12 / 30
Training: Elapsed time: 41530.2257 Loss: 3.4977 Acc: 0.2911
Epoch 13 / 30
Training: Elapsed time: 44710.9796 Lo

In [10]:
torch.save(full_model,'full_model_new_whale_as_extra_class.pt')